###  I followed two approaches with different order of functions for the prediction
###  Approach: 1) Prepocessed the data, removed Stopwords, Tokenization, Count Vectorization /TF-IDF, Scaling and 3 models (Logistic Regression, Naïve Bayes ) using GridsearchCV with Macro-f1 scoring
#### ---- I observed that the scoring was coming neared to 60% but not much greater than that

###  Approach: 2) Tokenization,Removed Stopwords, Preprocessing the data, Count Vectorization /TF-IDF, Scaling  with Normalization and 3 models (Logistic Regression, Naïve Bayes ) using GridsearchCV with Macro-f1 scoring
#### ---- I observed that the scoring improved by approximately 8% when compared with the above approach. 


In [50]:
# DataFrame
import pandas as pd
import spacy
# Matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import  SpaceTokenizer
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import PunktSentenceTokenizer

from nltk.tokenize import TreebankWordTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

# Set log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [27]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\14692\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
train=pd.read_csv("C:\\Users\\14692\\Documents\\Sem -III\\NLP\\train.csv",encoding='latin1')
train.head()

,Id,text,Target
0,1,@USAirways ! THE WORST in customer service. @...,-1
1,2,@united call wait times are over 20 minutes an...,-1
2,3,@JetBlue what's up with the random delay on fl...,-1
3,4,@AmericanAir Good morning! Wondering why my p...,0
4,5,@united UA 746. Pacific Rim and Date Night cut...,-1


In [29]:
upload_file=pd.read_csv("C:\\Users\\14692\\Documents\\Sem -III\\NLP\\test.csv",encoding='latin1')
upload_file.head()

,id,text
0,7322,@AmericanAir In car gng to DFW. Pulled over 1h...
1,7323,"@AmericanAir after all, the plane didnÂÃÂªt ..."
2,7324,@SouthwestAir can't believe how many paying cu...
3,7325,@USAirways I can legitimately say that I would...
4,7326,@AmericanAir still no response from AA. great ...


In [30]:
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from', 
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're',
             's', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [31]:
from nltk.stem import WordNetLemmatizer
def preprocess(textdata):
    processedText = []
    
    
    # Defining regex patterns.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern       = '@[^\s]+'
    alphaPattern      = "[^a-zA-Z0-9]"
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    numbers           = r"\d*"
    
    for tweet in textdata:
        tweet = tweet.lower()
        
        # Replace all URls with 'URL'
        tweet = re.sub(urlPattern,' ',tweet)
        # Replace all emojis.
        for emoji in emojis.keys():
            tweet = tweet.replace(emoji, emojis[emoji])        
        # Replace @USERNAME to 'USER'.
        tweet = re.sub(userPattern,' ', tweet)        
        # Replace all non alphabets.
        tweet = re.sub(alphaPattern, " ", tweet)
        # Replace 3 or more consecutive letters by 2 letter.
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)
        
        #Remove numbers
        tweet=re.sub(numbers,"",tweet)
        
        
            
        processedText.append(tweet)
        
        
    return processedText

In [51]:
traintext=train['text']
testtext=upload_file['text']
# traintext=(train['text']).apply(lambda x:preprocess(x))
# traintext=pd.DataFrame(traintext, columns=['text'])
traintext

0       @USAirways  ! THE WORST in customer service. @...
1       @united call wait times are over 20 minutes an...
2       @JetBlue what's up with the random delay on fl...
3       @AmericanAir Good morning!  Wondering why my p...
4       @united UA 746. Pacific Rim and Date Night cut...
                              ...                        
7315                              @AmericanAir followback
7316    @united thanks for the help. Wish the phone re...
7317    @usairways the. Worst. Ever. #dca #customerser...
7318    @nrhodes85: look! Another apology. DO NOT FLY ...
7319    @united you are by far the worst airline. 4 pl...
Name: text, Length: 7320, dtype: object

In [33]:
def removestopwords(data):
    words_only = re.sub("[^a-zA-Z]"," ",str(data))       

    words = words_only.lower().split()     
    stops = set(stopwords.words("english"))      
    tokenize_words = [w for w in words if not w in stops]      
    return (" ".join(tokenize_words))

In [34]:
def RemoveStopwords(data):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = data.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)

In [35]:
traintext1=list(map(RemoveStopwords,traintext))
testtext1=list(map(RemoveStopwords,testtext))

In [37]:
#divided the set into 80:20 ratio train-test set
X_train=(traintext1[0:int(len(train)*0.8)])
y_train=train['Target'][0:int(len(train)*0.8)]
X_test=(traintext1[int(len(train)*0.8):])
y_test=train['Target'][int(len(train)*0.8):]

In [38]:
def read_data(file, tokenizer):
    corpus=[]
    for sentence in file:
        tokens=' '.join(tokenizer(sentence))
        corpus.append(tokens)
    return corpus


In [39]:
# spaCy lemmatization needs tagger but disable the rest
nlp = spacy.load('en_core_web_sm', disable=['ner,parser'])
nlp.remove_pipe('ner')
nlp.remove_pipe('parser')


def get_part_of_speech_tags(token):
    
    """Maps POS tags to first character lemmatize() accepts.
    We are focussing on Verbs, Nouns, Adjectives and Adverbs here."""

    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    tag = nltk.pos_tag([token])[0][1][0].upper()
    
    return tag_dict.get(tag, wordnet.NOUN)


In [40]:
def tweet_tokenizer(data):
    tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
    text=tknzr.tokenize(data)
    return text

# def punkt_tokenizer(data):
#     sent_tokenizer = PunktSentenceTokenizer(data)
#     return sent_tokenizer
    
def whitespace_tokenizer(data):
    return str.split(data)

def potter_tokenizer(data):
    tokenizer= potter_tokenizer()
    return tokenizer.tokenize(data)

def spacy_tokenizer(data):
    spacy_tokens=nlp(data)
    return [token.text for token in spacy_tokens]

def spacy_lemmatizer(data):
    spacy_tokens=nlp(data)
    return [token.lemma_ for token in spacy_tokens]

def nltk_treebank_tokenizer(data):
    tokenizer= TreebankWordTokenizer()
    return tokenizer.tokenize(data)

def nltk_treebank_stemmer_tokenizer(data):
    tokenizer= TreebankWordTokenizer()
    stemmer = PorterStemmer()
    tokens= tokenizer.tokenize(data)
    return [stemmer.stem(token) for token in tokens]

def nltk_treebank_lemmatizer_tokenizer(data):
    tokenizer= TreebankWordTokenizer()
    lemmatizer = WordNetLemmatizer()  
    tokens= tokenizer.tokenize(data)
    return [lemmatizer.lemmatize(token, get_part_of_speech_tags(token)) for token in tokens]

In [ ]:
def evaluate_tokenizer(tokenizer, X_train, Y_train,X_test, Y_test, cv):


    X_train=read_data(X_train, tokenizer)
    X_test=read_data(X_test, tokenizer)

    vectorizer = CountVectorizer(max_features=10000, analyzer=str.split, lowercase=False, strip_accents=None, binary=True)
    X_train_cv = vectorizer.fit_transform(X_train)
    X_test_cv = vectorizer.transform(X_test)

    scaler = preprocessing.StandardScaler(with_mean= False).fit(X_train_cv)
    X_train_cv=scaler.transform(X_train_cv) #Scaling the data
    X_test_cv=scaler.transform(X_test_cv) #Scaling the data

    vectoriser = TfidfVectorizer(ngram_range=(1,1), max_features=60000,use_idf=True, smooth_idf=True)
    X_train_tf=vectoriser.fit_transform(X_train)
    X_test_tf=vectoriser.transform(X_test)

    scaler = preprocessing.StandardScaler(with_mean= False).fit(X_train_tf)
    X_train_tf=scaler.transform(X_train_tf) #Scaling the data
    X_test_tf=scaler.transform(X_test_tf) #Scaling the data

    #fitting the model for Count Vectorizer
#         logreg_cv = linear_model.LogisticRegression(C=1.0,max_iter=1000, penalty='l2')
#         logreg_cv.fit(X_train_cv, Y_train)
#         y_pred_cv=logreg_cv.predict(X_test_cv)

    parameters={'C': [2,3,4], 'max_iter': [1000], 'n_jobs': [-1], 'penalty': ['l2']}
    grid_cv = GridSearchCV(LogisticRegression(), param_grid=parameters, cv=cv)
    grid_cv.fit(X_train_cv, Y_train)
    y_pred_cv = grid_cv.predict(X_test_cv)
    print("Macro f score of %s for Count Vectorizer is %f" % (tokenizer.__name__, f1_score(Y_test, y_pred_cv, average='macro')))

    #fitting the model for TF-IDF
#         logreg_tf = linear_model.LogisticRegression(C=1.0,max_iter=1000, penalty='l2')
#         logreg_tf.fit(X_train_tf, Y_train)
#         y_pred_tf=logreg_tf.predict(X_test_tf)
    grid_tf = GridSearchCV(LogisticRegression(), param_grid=parameters, cv=cv)
    grid_tf.fit(X_train_tf, Y_train)
    y_pred_tf = grid_tf.predict(X_test_tf)
    print("Macro f score %s for TF-IDF is %f" % (tokenizer.__name__, f1_score(Y_test, y_pred_tf, average='macro')))


In [171]:
#for cv=5
tokenizers= [whitespace_tokenizer, spacy_tokenizer, nltk_treebank_tokenizer, nltk_treebank_stemmer_tokenizer, nltk_treebank_lemmatizer_tokenizer]
for tokenizer in tokenizers:
    print(evaluate_tokenizer(tokenizer, X_train, y_train,X_test, y_test, 5))

Macro f score of whitespace_tokenizer for Count Vectorizer is 0.588467
Macro f score whitespace_tokenizer for TF-IDF is 0.576487
None
Macro f score of spacy_tokenizer for Count Vectorizer is 0.587798
Macro f score spacy_tokenizer for TF-IDF is 0.574157
None
Macro f score of nltk_treebank_tokenizer for Count Vectorizer is 0.589661
Macro f score nltk_treebank_tokenizer for TF-IDF is 0.576454
None
Macro f score of nltk_treebank_stemmer_tokenizer for Count Vectorizer is 0.600214
Macro f score nltk_treebank_stemmer_tokenizer for TF-IDF is 0.585200
None
Macro f score of nltk_treebank_lemmatizer_tokenizer for Count Vectorizer is 0.608647
Macro f score nltk_treebank_lemmatizer_tokenizer for TF-IDF is 0.588033
None


In [172]:
#for cv=3
tokenizers= [whitespace_tokenizer, spacy_tokenizer, nltk_treebank_tokenizer, nltk_treebank_stemmer_tokenizer, nltk_treebank_lemmatizer_tokenizer]
for tokenizer in tokenizers:
    print(evaluate_tokenizer(tokenizer, X_train, y_train,X_test, y_test,3))

Macro f score of whitespace_tokenizer for Count Vectorizer is 0.588467
Macro f score whitespace_tokenizer for TF-IDF is 0.576487
None
Macro f score of spacy_tokenizer for Count Vectorizer is 0.587798
Macro f score spacy_tokenizer for TF-IDF is 0.574157
None
Macro f score of nltk_treebank_tokenizer for Count Vectorizer is 0.589661
Macro f score nltk_treebank_tokenizer for TF-IDF is 0.576454
None
Macro f score of nltk_treebank_stemmer_tokenizer for Count Vectorizer is 0.600214
Macro f score nltk_treebank_stemmer_tokenizer for TF-IDF is 0.585200
None
Macro f score of nltk_treebank_lemmatizer_tokenizer for Count Vectorizer is 0.608647
Macro f score nltk_treebank_lemmatizer_tokenizer for TF-IDF is 0.588033
None


# Now changing the order of execution of functions 

In [52]:
def evaluate_tokenizer(tokenizer, X_train, Y_train,X_test, Y_test, cv):

    #Tokenization
    X_train=read_data(X_train, tokenizer)
    X_test=read_data(X_test, tokenizer)

    #Again Remove Stopwords
    X_train=list(map(RemoveStopwords,X_train))
    X_test=list(map(RemoveStopwords,X_test))
    
    #Preprocessing
    X_train=preprocess(X_train)
    X_test=preprocess(X_test)

    vectorizer = CountVectorizer(max_features=10000, analyzer=str.split, lowercase=False, strip_accents=None, binary=True)
    X_train_cv = vectorizer.fit_transform(X_train)
    X_test_cv = vectorizer.transform(X_test)

    scaler = preprocessing.StandardScaler(with_mean= False).fit(X_train_cv)
    X_train_cv=scaler.transform(X_train_cv) #Scaling the data
    X_test_cv=scaler.transform(X_test_cv) #Scaling the data
    
    #Normalize
    X_train_cv = preprocessing.normalize(X_train_cv)
    X_train_cv = preprocessing.normalize(X_train_cv)
    
    vectoriser = TfidfVectorizer(ngram_range=(1,1), max_features=60000,use_idf=True, smooth_idf=True)
    X_train_tf=vectoriser.fit_transform(X_train)
    X_test_tf=vectoriser.transform(X_test)

    scaler = preprocessing.StandardScaler(with_mean= False).fit(X_train_tf)
    X_train_tf=scaler.transform(X_train_tf) #Scaling the data
    X_test_tf=scaler.transform(X_test_tf) #Scaling the data

    #Normalize
    X_train_tf = preprocessing.normalize(X_train_tf)
    X_test_tf = preprocessing.normalize(X_test_tf)

    #fitting the models for Count Vectorizer        
    parameters={'C': [1,2,3,4], 'max_iter': [3000], 'warm_start': [True],'n_jobs': [-1], 'penalty': ['l2']}
    grid_cv = GridSearchCV(LogisticRegression(), param_grid=parameters, cv=cv)
    grid_cv.fit(X_train_cv, Y_train)
    y_pred_cv = grid_cv.predict(X_test_cv)
    print(" Logistic Macro-f score of %s for Count Vectorizer is %f" % (tokenizer.__name__, f1_score(Y_test, y_pred_cv, average='macro')))

    parameters={'priors':[None], 'var_smoothing':[1e-09]}
    grid_cv = GridSearchCV(GaussianNB(), param_grid=parameters, cv=cv)
    grid_cv.fit(X_train_cv.toarray(), Y_train)
    y_pred_cv = grid_cv.predict(X_test_cv.toarray())
    print("Naive Bayes Macro-f score of %s for Count Vectorizer is %f" % (tokenizer.__name__, f1_score(Y_test, y_pred_cv, average='macro')))

    parameters={'max_depth':[2,4,5],'n_estimators':[200,400], 'learning_rate':[1]}
    grid_cv = GridSearchCV(XGBClassifier(), param_grid=parameters, cv=cv)
    grid_cv.fit(X_train_cv, Y_train)
    y_pred_cv = grid_cv.predict(X_test_cv)
    print("XGBoost Macro-f score of %s for Count Vectorizer is %f" % (tokenizer.__name__, f1_score(Y_test, y_pred_cv, average='macro')))

    ########################################################################################
    #fitting the models for TF-IDF
    parameters={'C': [1,2,3,4], 'max_iter': [3000], 'warm_start': [True], 'n_jobs': [-1], 'penalty': ['l2']}
    grid_tf = GridSearchCV(LogisticRegression(), param_grid=parameters, cv=cv)
    grid_tf.fit(X_train_tf, Y_train)
    y_pred_tf = grid_tf.predict(X_test_tf)
    print("Logistic Macro-f score %s for TF-IDF is %f" % (tokenizer.__name__, f1_score(Y_test, y_pred_tf, average='macro')))

    parameters={'priors':[None], 'var_smoothing':[1e-09]}
    grid_tf = GridSearchCV(GaussianNB(), param_grid=parameters, cv=cv)
    grid_tf.fit(X_train_tf.toarray(), Y_train)
    y_pred_tf = grid_tf.predict(X_test_tf.toarray())
    print("Naive Bayes Macro-f score of %s for TF-IDF is %f" % (tokenizer.__name__, f1_score(Y_test, y_pred_tf, average='macro')))

    parameters={'max_depth':[2,4,5],'n_estimators':[200,400],'learning_rate':[1]}
    grid_tf = GridSearchCV(XGBClassifier(), param_grid=parameters, cv=cv)
    grid_tf.fit(X_train_tf, Y_train)
    y_pred_tf = grid_tf.predict(X_test_tf)
    print("XGBoost Macro-f score of %s for TF-IDF is %f" % (tokenizer.__name__, f1_score(Y_test, y_pred_tf, average='macro')))


In [53]:
#for cv=2
tokenizers= [tweet_tokenizer, whitespace_tokenizer, spacy_tokenizer, nltk_treebank_tokenizer, nltk_treebank_stemmer_tokenizer, nltk_treebank_lemmatizer_tokenizer]
for tokenizer in tokenizers:
    print(evaluate_tokenizer(tokenizer, X_train, y_train,X_test, y_test,2))

 Logistic Macro-f score of tweet_tokenizer for Count Vectorizer is 0.643350
Naive Bayes Macro-f score of tweet_tokenizer for Count Vectorizer is 0.554394
XGBoost Macro-f score of tweet_tokenizer for Count Vectorizer is 0.632155
Logistic Macro-f score tweet_tokenizer for TF-IDF is 0.603429
Naive Bayes Macro-f score of tweet_tokenizer for TF-IDF is 0.490938
XGBoost Macro-f score of tweet_tokenizer for TF-IDF is 0.705212
None
 Logistic Macro-f score of whitespace_tokenizer for Count Vectorizer is 0.642393
Naive Bayes Macro-f score of whitespace_tokenizer for Count Vectorizer is 0.558992
XGBoost Macro-f score of whitespace_tokenizer for Count Vectorizer is 0.640975
Logistic Macro-f score whitespace_tokenizer for TF-IDF is 0.609373
Naive Bayes Macro-f score of whitespace_tokenizer for TF-IDF is 0.489211
XGBoost Macro-f score of whitespace_tokenizer for TF-IDF is 0.715166
None
 Logistic Macro-f score of spacy_tokenizer for Count Vectorizer is 0.644218
Naive Bayes Macro-f score of spacy_token

#### XGBoost with whitespace_tokenizer performs the best among all the models with combinations

## Predict the Target value of test data from the above results  

In [63]:
def evaluate_tokenizer(tokenizer, X_train, Y_train,X_test, cv):
        
   #Tokenization
    X_train=read_data(X_train, tokenizer)
    X_test=read_data(X_test, tokenizer)
    
    #Preprocessing
    traintext=preprocess(X_train)
    testtext=preprocess(X_test)

#     #Again Remove Stopwords
#     X_train=list(map(RemoveStopwords,traintext))
#     X_test=list(map(RemoveStopwords,testtext))

    vectorizer = CountVectorizer(max_features=10000, analyzer=str.split, lowercase=False, strip_accents=None, binary=True)
    X_train_cv = vectorizer.fit_transform(X_train)
    X_test_cv = vectorizer.transform(X_test)

    scaler = preprocessing.StandardScaler(with_mean= False).fit(X_train_cv)
    X_train_cv=scaler.transform(X_train_cv) #Scaling the data
    X_test_cv=scaler.transform(X_test_cv) #Scaling the data

    #Normalize
    X_train_cv = preprocessing.normalize(X_train_cv)
    X_train_cv = preprocessing.normalize(X_train_cv)        
        
    parameters={'C': [0.001,0.01,0.1,1,2,4], 'max_iter': [2000,3000],'n_jobs': [-1], 'penalty': ['l2','newton-cg']}
    grid_cv = GridSearchCV(LogisticRegression(), param_grid=parameters, cv=cv, n_jobs=-1)
    grid_cv.fit(X_train_cv, Y_train)
    y_pred_cv = grid_cv.predict(X_test_cv)
    return y_pred_cv

In [64]:
y_pred=evaluate_tokenizer(nltk_treebank_tokenizer, traintext, train['Target'],testtext, 10)
y_pred

array([-1,  0, -1, ..., -1,  1, -1], dtype=int64)

In [65]:
# upload_file['target']=y_pred
# upload_file.to_csv (r'C:\\Users\\14692\\Documents\\Sem -III\\NLP\\predicted_final.csv', index = False, header=True)